In [7]:
#!pip install snowflake-snowpark-python
#!pip install snowflake-ml-python

In [8]:
import pandas as pd
import json
import os
import snowflake.snowpark.functions as F
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions
from snowflake.snowpark import Session

In [9]:
connection_parameters = {
    "account": os.getenv("SNOWFLAKE_ACCOUNT"),
    "user": os.getenv("SNOWFLAKE_USER"),
    "password": os.getenv("SNOWFLAKE_PASSWORD"),
    "schema": os.getenv("SNOWFLAKE_SCHEMA"),
    "database": os.getenv("SNOWFLAKE_DATABASE"),
    "role": os.getenv("SNOWFLAKE_ROLE"),
    "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE"),
}

session = Session.builder.configs(connection_parameters).create()

In [10]:
session.sql("CREATE TRANSIENT  DATABASE IF NOT EXISTS THINGSBOARD;").collect()
session.sql("CREATE  TRANSIENT  SCHEMA IF NOT EXISTS THINGSBOARD.SENSOR;").collect()

session.sql("USE DATABASE THINGSBOARD").collect()
session.sql("USE SCHEMA THINGSBOARD.SENSOR").collect()

[Row(status='Statement executed successfully.')]

In [12]:
session.sql("""
    CREATE OR REPLACE TRANSIENT TABLE THINGSBOARD.SENSOR.Environmental 
    (ts TIMESTAMP,
     device VARCHAR, 
     co FLOAT,
     humidity FLOAT, 
     light BOOLEAN, 
     lpg FLOAT,
     motion BOOLEAN, 
     smoke FLOAT, 
     temp FLOAT)
""").collect()

[Row(status='Table ENVIRONMENTAL successfully created.')]

[Row(TS=None, DEVICE=None, CO=None, HUMIDITY=None, LIGHT=None, LPG=None, MOTION=None, SMOKE=None, TEMP=None),
 Row(TS=None, DEVICE=None, CO=None, HUMIDITY=None, LIGHT=None, LPG=None, MOTION=None, SMOKE=None, TEMP=None)]

In [29]:

import os

aws_access_key = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_key = os.getenv('AWS_SECRET_ACCESS_KEY')

# Create stage with env variables
session.sql(f"""
    CREATE OR REPLACE STAGE THINGSBOARD.SENSOR.s3_stage
    url = 's3://fastapi-snowflake/'
    credentials=(AWS_KEY_ID='{aws_access_key}' AWS_SECRET_KEY='{aws_secret_key}')
    file_format = (type = 'JSON');
""").collect()

# Copy data using the stage
session.sql("""
    COPY INTO THINGSBOARD.SENSOR.Environmental
    FROM @THINGSBOARD.SENSOR.s3_stage/Environmental.json
    FILE_FORMAT = (TYPE = JSON)
    MATCH_BY_COLUMN_NAME = CASE_SENSITIVE;
""").collect()


[Row(status='Copy executed with 0 files processed.')]

In [32]:
session.sql("SELECT * FROM THINGSBOARD.SENSOR.Environmental").collect()

[Row(TS=None, DEVICE=None, CO=None, HUMIDITY=None, LIGHT=None, LPG=None, MOTION=None, SMOKE=None, TEMP=None),
 Row(TS=None, DEVICE=None, CO=None, HUMIDITY=None, LIGHT=None, LPG=None, MOTION=None, SMOKE=None, TEMP=None)]